In [ ]:
import requests
import json
import requests
from PIL import Image
from io import BytesIO
from fpdf import FPDF
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from openpyxl import load_workbook
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


all_restaurants = []


def get_hotel_data(city_name):

    for p_nunber in range(93):
    
        print(p_nunber)
        print("#--------------------------------------------------------------------------------------------------------------------") 

        url = f'https://www.dineout.co.in/surat-restaurants?search_str={city_name}'+"&p=" + str(p_nunber)
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
        }

        # Fetch the search page
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
            return None

        soup = BeautifulSoup(response.text, 'html.parser')

        data_links = [element["data-link"] for element in soup.find_all(attrs={"data-link": True})]
    
        data_links = set(data_links)


        
        for link in data_links:
            hotel_url = "https://www.dineout.co.in" + link
            print(f"Processing restaurant: {hotel_url}")
            try:
                details = get_data(hotel_url)
                if details:
                    all_restaurants.append(details)  # Append restaurant details to the list
            except Exception as e:
                print(f"Error processing {hotel_url}: {e}")
    

def create_pdf_from_images(image_urls, output_file):
    # Initialize FPDF
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)

    for i, url in enumerate(image_urls, start=1):
        try:
            # Fetch the image
            response = requests.get(url)
            response.raise_for_status()  # Raise an exception for HTTP errors

            # Open the image using Pillow
            img = Image.open(BytesIO(response.content))
            img_path = f"temp_image_{i}.jpg"  # Temporary file to save the image

            # Save the image locally
            img.save(img_path, "JPEG")

            # Add the image to the PDF
            pdf.add_page()
            pdf.image(img_path, x=10, y=10, w=190)  # Adjust dimensions as needed

        except Exception as e:
            print(f"Failed to process image {url}: {e}")

    # Save the PDF to the specified output file
    pdf.output(output_file)
    print(f"PDF saved as {output_file}")


def save_to_excel(data, filename):
    print("Saving data to Excel...")
    df = pd.DataFrame(data)  # Convert list of dictionaries to a DataFrame
    df.to_excel(filename, index=False)  # Save to Excel without the index column
    print(f"Data saved to {filename}.")

# -------------------------------- Selenium for Dynamic Content --------------------------------

def get_data(hotel_url):
    # Initialize Selenium WebDriver
    service = Service(r'D:\Company\chromedriver.exe')  # Update the path if needed
    driver = webdriver.Chrome(service=service)

    try:
        # Open the hotel page
        driver.get(hotel_url)

        # Wait for the menu gallery to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "demo-gallery"))
        )

        # Extract page source after rendering
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        # Extract the <ul> element containing the menu
        menu_gallery = soup.find('ul', class_='d-flex demo-gallery')
        menu_links = []
        if menu_gallery:
            for link in menu_gallery.find_all('a'):
                href = link.get('href')
                if href:
                    menu_links.append(href)

        # Extract "restnt-info" details
        details_section = soup.find('div', class_='restnt-details_info')
        restaurant_details = {}
        
        name = "null"
        location = "null"
        cost_for_two = "null"
    
        name = details_section.find('h1').text.strip()
        cost_for_two = details_section.find('div', class_='restnt-cost').text.split('|')[0].strip()
         
        type_header = soup.find('h4', string="TYPE")  
        type = "null"
        if type_header:
            type = type_header.find_next_sibling('p').text.strip()  
        
        type_header_2 = soup.find('h5', string="Call the restaurant")
        moblie_number = "null"  
        if type_header_2:
            moblie_number = type_header_2.find_next_sibling('p').text.strip() 

        type_header_3 = soup.find('h3', string="Address:") 
        if type_header_3:
            location = type_header_3.find_next_sibling('p').text.strip()
                
        # Look for the div containing the rating dynamically
        rating_div = soup.find('div', class_=lambda c: c and 'rest-rating' in c)
        
        rating = "null"
        if rating_div:
            rating = rating_div.text.strip()
                  

        restaurant_details = {
            "name": name,
            "location": location,
            "cost_for_two": cost_for_two,  
            "rating": rating,
            "type": type,
            "moblie_number": moblie_number 
        }

        restaurant_details["type"] = "Dineout Pay" in restaurant_details["type"]

        

        cost_for_two = cost_for_two.split(' ')[1].strip()  # Extract the numeric part
        restaurant_details["cost_for_two"] = cost_for_two
        
        file_menu_name = name + "'s menu.pdf"
        create_pdf_from_images(menu_links, file_menu_name)

        return restaurant_details

 

    finally:
        # Quit Selenium WebDriver
        driver.quit()


In [26]:
city_name = "Surat"#input("Enter the hotel name: ") 
get_hotel_data(city_name)

0
#--------------------------------------------------------------------------------------------------------------------
Processing restaurant: https://www.dineout.co.in/surat/mr-multicuisine-restaurant-and-banquets-piplod-south-surat-100733
Processing restaurant: https://www.dineout.co.in/surat/the-dango-cake-pal-gam-west-surat-111946
Processing restaurant: https://www.dineout.co.in/surat/kurtosshhh-vesu-south-surat-62362
Processing restaurant: https://www.dineout.co.in/surat/leonardo-italian-mediterranean-dining-piplod-south-surat-60233
Processing restaurant: https://www.dineout.co.in/surat/atul-bakery-hazira-west-surat-117202
Processing restaurant: https://www.dineout.co.in/surat/sizzling-bite-piplod-south-surat-60255
Processing restaurant: https://www.dineout.co.in/surat/spice-villa-piplod-south-surat-60239
Processing restaurant: https://www.dineout.co.in/surat/masala-stories-vesu-south-surat-97434
Processing restaurant: https://www.dineout.co.in/surat/steam-dumas-south-surat-59518


KeyboardInterrupt: 

In [27]:
save_to_excel(all_restaurants, "restaurants_data.xlsx")
print("Scraping completed. Data saved to 'restaurants_data.xlsx'.")

Saving data to Excel...
Data saved to restaurants_data.xlsx.
Scraping completed. Data saved to 'restaurants_data.xlsx'.
